---
title: Polymorphism and Inheritance
abstract: |
    Inheritance allows derived classes to reuse the functionality of their base classes. Through base-class pointers or references, derived class instances can be treated as base class objects, enabling polymorphism. To ensure safe polymorphic deletion, the base class must declare a virtual destructor. Classes that manage dynamic resources should implement a destructor, copy constructor, and copy assignment operator—collectively known as the Rule of Three. For efficient resource transfer, they should also define a move constructor and move assignment operator, completing the Rule of Five.
skip_execution: true
---

In [ ]:
from __init__ import *
!mkdir -p private

In [ ]:
if not input('Load JupyterAI? [Y/n]').lower()=='n':
    %reload_ext jupyter_ai

## Motivation

Recall the implementation of `Counter` class:

In [ ]:
%%cpp
class Counter {
    int count=0;
public:
    Counter() = default;
    explicit Counter(int count) : count{count} {}
    int operator()() {
        return ++count;
    }
    void set_count(int count) { 
        if (count>=0) this->count=count; 
        else cerr << "Count cannot go negative.\n";  
    }
    operator int() const {
        return count;
    }
};

Counter counter1;
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';
counter1.set_count(10);
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';

We can modify the code as follows to count backwards:

In [ ]:
%%cpp
class Counter {
    int count;
public:
    Counter() = delete;
    explicit Counter(int count) : count{count} {}
    int operator()() {
        return set_count(count-1), count;
    }
    void set_count(int count) {
        if (count>=0) this->count=count; 
        else cerr << "Count cannot go negative.\n";  
    }
    operator int() const {
        return count;
    }
};

// Counter counter2; // error: call to deleted constructor of 'Counter'
Counter counter2(3);
for (auto i=0; i<5; i++) cout << counter2() << ' '; cout << '\n';

Unlike the forward counter:

1. The backward counter has the default constructor deleted using the [`delete` keyword](https://en.cppreference.com/w/cpp/language/default_constructor.html#Deleted_default_constructor); and
2. the `operator()` calls `set_count` to increment the `count`.

::::{exercise}
:label: ex:backward_counter
Explain the motivations for the changes.

::::

YOUR ANSWER HERE

Both the forward and backward counters have the same implementation for the explicit constructor, the `set_count` methd, and the converting operator `int`. How to avoid code duplication?

::::{exercise}
:label: ex:counter_with_step

One way is to introduce a new property called `step` that can takes the value `1` for the forward counter, or `-1` for the backward counter. Complete for following code for such an implementation.

::::

In [ ]:
%%cpp
class Counter {
    int count=0, step=1;
public:
    Counter() = default;
    explicit Counter(int count) : count{count} {}
    Counter(int count, int step) : count{count}, step(step) {}
    int operator()() {
        /*
        # REPLACE THE ENTIRE COMMENT WITH YOUR CODE #
        */
    }
    void set_count(int count) { 
        if (count>=0) this->count=count; 
        else cerr << "Count cannot go negative.\n";
    }
    operator int() const {
        return count;
    }
};

// tests
Counter counter1, counter2(3, -1);
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';
for (auto i=0; i<5; i++) cout << counter2() << ' '; cout << '\n';
/* Expected
1 2 3 4 5 
2 1 0 0 0 
Count cannot go negative.
Count cannot go negative.
*/

There is a slight inefficiency that `operator()()` needs to check if the new count is non-negative even when the step size is positive. If we want to implement more features specific for backward counter, we would have to worry about how to merge them with the forward counter. The complexity increases quickly when we want to introduce other kinds counters later, and code may not be backward compatible or efficient for the previous versions.

A better solution is to simply define different kinds of counters as separate derived classes of a common base class:
- Members of the base class can be inherited, allowing code reuse.
- Derived classes can override/overload members of its base classes, resulting in polymorphism.

## Derived Class

### Inheritance

The following defines the forward counter as a base class:

In [ ]:
%%cpp
class Counter {
protected: // instead of private
    int count;
public:
    Counter() = default;
    explicit Counter(int count) : count{count} {}
    int operator()() {
        return ++count;
    };
    void set_count(int count) {
        if (count>=0) this->count=count; 
        else cerr << "Count cannot go negative.\n";
    }
    operator int() const {
        return count;
    }
};

Counter counter1;
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';

This is the same as the previous implementation except that `count` now has [`protected` member access](https://en.cppreference.com/w/cpp/language/access.html#Protected_member_access), which make it accessible to members and friends of derived classes.

The following defines `BackwardCounter` as a subclass of the base class `Counter`:

In [ ]:
%%cpp
class BackwardCounter : public Counter {
public:
    BackwardCounter() = delete;
    explicit BackwardCounter(int count) : Counter(count) {}
    int operator()() {
        set_count(count - 1);
        return count;
    }
};

// BackwardCounter counter2; // error: call to deleted constructor of 'BackwardCounter'
BackwardCounter counter2(3);
for (auto i=0; i<5; i++) cout << counter2() << ' '; cout << '\n';

The class declaration `class BackwardCounter : public Counter` has a [base clause](https://en.cppreference.com/w/cpp/language/derived_class.html) that specifies the list of base classes.
- `public` member specifier ([public inheritance](https://en.cppreference.com/w/cpp/language/derived_class.html#Public_inheritance)) for a base class means the access to the members of the base class is also inherited.
- `protected` member specifier ([protected inheritance](https://en.cppreference.com/w/cpp/language/derived_class.html#Protected_inheritance)) means that `public` members of the base clas become `protected`.
- `private`  member specifier ([private inheritance](https://en.cppreference.com/w/cpp/language/derived_class.html#Private_inheritance)) means that both the `public` and `protected` members of the base class become `private`.

The constructor `explicit BackwardCounter(int count) : Counter(count)` has an initialization list that calls the base class's constructor `Counter(count)`. [Base classes are initialized before members](https://en.cppreference.com/w/cpp/language/initializer_list.html#Initialization_order).

::::{exercise}
:label:  ex:inheriting_constructors

Explain how the constructors are defined in the following derived class.

:::{hint}

See the [inheriting constructors](https://en.cppreference.com/w/cpp/language/using_declaration.html#Inheriting_constructors).

:::

::::

In [ ]:
%%cpp
class BackwardCounter : public Counter {
public:
    BackwardCounter() = delete;
    using Counter::Counter;
    int operator()() {
        set_count(count - 1);
        return count;
    }
};

BackwardCounter counter2(3);
for (auto i=0; i<5; i++) cout << counter2() << ' '; cout << '\n';

YOUR ANSWER HERE

### Polymorphism

Can we treat `BackwardCounter` as a `Counter`? That would be convenient, especially when working with multiple counters:

In [ ]:
%%cpp
vector<Counter> counters { Counter(), BackwardCounter(3) };
for (auto &counter: counters) {
    for (auto i=0; i<5; i++) cout << counter() << ' '; cout << '\n';
}

::::{exercise}
:label: ex:pm1

Unfortunately, the above code does not work properly. Why?

::::

::::{solution} ex:pm1
:class: dropdown

`BackCounter(3)` is converted to `Counter` so it counts up instead of down because the compiler dispatch statically to the `operator()` method implemented in `Counter` instead of `BackCounter`.

::::

To properly override a method even if there is no compile-time information about the actual type of the class, the method should be marked using the [`virtual` function specifier](https://en.cppreference.com/w/cpp/language/virtual.html) in the base class:

In [ ]:
%%cpp
class Counter {
protected:
    int count;
public:
    Counter(int count=0) : count{count} {}
    virtual ~Counter() = default; // allow polymorphic deletion

    virtual int operator()() {  // can be overridden
        return ++count;
    }
    void set_count(int count) {
        if (count>=0) this->count=count; 
        else cerr << "Count cannot go negative.\n";
    }
    operator int() const {
        return count;
    }
};

::::{caution} Virtual destructor

For any class that is meant to be inherited, it is a good practice to make the destructor 
- public and virtual so that it can be properly overriden by the derived class to clean up the resources of the derived class; or
- protected and non-virtual so that the base class cannot be used for destruction.

On the contrary, virtual constructor is not allowed in C++.

::::

In [ ]:
%%ai
Explain in a paragraph why virtual constructor is not allowed in C++.

The code for `BackwardCounter` is much reduced:

In [ ]:
%%cpp
class BackwardCounter : public Counter {
public:
    BackwardCounter() = delete;
    explicit BackwardCounter(int count) : Counter(count) {}
    virtual int operator()() override {
        set_count(count - 1);
        return count;
    }
};

::::{caution}

It is good practice to add the `override` qualifier to let the compiler know it is intended to override the base version. If the compiler cannot find a base version with the same signature, which may happen due to typos, the compiler will raise an error instead of creating a new overload.

::::

It seems to work now:

In [ ]:
%%cpp
BackwardCounter counter1(3);
Counter *counter2=&counter1;
for (auto i=0; i<5; i++) cout << (*counter2)() << ' '; cout << '\n';
Counter &counter3=counter1;
for (auto i=0; i<5; i++) cout << counter3() << ' '; cout << '\n';

Even though `*counter2` and `counter3` have type `Counter` instead of `BackwardCounter`, they both use `operator()` defined in `BackwardCounter`. This is achieved using [dynamic dispatch](https://en.wikipedia.org/wiki/Dynamic_dispatch) for virtual functions.

In [ ]:
%%ai
Explain in a paragraph or two how C++ implements dynamic dispatch for 
virtual functions. Why the name virtual?

Note also that there is no need use perform type casting when converting the address of a derived class to that of the base class. In comparison, [`reinterpret_cast`](https://en.cppreference.com/w/cpp/language/reinterpret_cast.html) is required for the reverse conversion:

In [ ]:
%%cpp
Counter counter1;
BackwardCounter *counter2=reinterpret_cast<BackwardCounter *>(&counter1);
for (auto i=0; i<5; i++) cout << (*counter2)() << ' '; cout << '\n';
BackwardCounter &counter3=reinterpret_cast<BackwardCounter &>(counter1);
for (auto i=0; i<5; i++) cout << counter3() << ' '; cout << '\n';

Such conversion is unusual, however, since `Counter` does not initialize any new members introduced in `BackwardCounter`.

::::{exercise}
:label: ex:vptr

Explain whether `*counter2` and `counter3` of type `BackwardCounter` counts backward.

::::

YOUR ANSWER HERE

### Base Class Pointers

Unfortunately, the issue remains even after using `virtual` function specifier:

In [ ]:
%%cpp
vector<Counter> counters { Counter(), BackwardCounter(3) };
for (auto &counter: counters) {
    for (auto i=0; i<5; i++) cout << counter() << ' '; cout << '\n';
}

::::{caution} Why doesn't `vector<Counter>` allow polymorphism?
:class: dropdown

The issue is that `BackwardCounter(3)` is sliced down to a `Counter` object when copied/moved/cast to `Counter`, as illustrated in the following code.

::::

In [ ]:
%%cpp
BackwardCounter counter1(3);
Counter counter2=counter1;            // copied
for (auto i=0; i<5; i++) cout << counter2() << ' '; cout << '\n';
Counter counter3=std::move(counter1); // moved
for (auto i=0; i<5; i++) cout << counter3() << ' '; cout << '\n';
cout << static_cast<Counter>(counter1)() << '\n';

To fix the issue, we should use `std::vector<Counter *>` instead of `std::vector<Counter>`:

In [ ]:
%%cpp
Counter counter1;
BackwardCounter counter2(3);
vector<Counter *> counters { &counter1, &counter2 };
for (auto counter: counters) {
    for (auto i=0; i<5; i++) cout << (*counter)() << ' '; cout << '\n';
}

::::{important} Base class pointers for polymorphism

`counter2` is said to be accessed through its base class pointer, which only reinterpret but not modify the object it points to.

::::

::::{exercise}
:label: ex:vector_of_pointers

How to avoid giving names to each element? Modify the code to create the counters in the heap.

::::

In [ ]:
%%cpp
{
    /*
    # REPLACE THE ENTIRE COMMENT WITH YOUR CODE #
    */
    for (auto &counter: counters) {
        for (auto i=0; i<5; i++) cout << (*counter)() << ' '; cout << '\n';
        delete counter;  // avoid memory leak
        counter=nullptr; // avoid double deletion
    }
    /* Expected
    1 2 3 4 5 
    2 1 0 0 0 
    Count cannot go negative.
    Count cannot go negative.
    */
} // counters out of scope

To avoid releasing the memory manually, we can use smart pointers such as [`std::unique_ptr`](https://en.cppreference.com/w/cpp/memory/unique_ptr.html) in `<memory>`:

In [ ]:
%%cpp
Counter *count1;
{
    // vector<unique_ptr<Counter>> counters { make_unique<Counter>(), make_unique<BackwardCounter>(3) }; // fails as unique_ptr is non-copyable
    vector<unique_ptr<Counter>> counters;
    counters.push_back(make_unique<Counter>());
    counters.push_back(make_unique<BackwardCounter>(3));
    
    for (auto& counter : counters) {
        for (int i = 0; i < 5; ++i) cout << (*counter)() << ' '; cout << '\n';
    }
} // members of counters destroyed automatically

To create copyable smart pointers, we can instead use the smart pointer [`std::unique_ptr`](https://en.cppreference.com/w/cpp/memory/shared_ptr.html) in `<memory>`:

In [ ]:
%%cpp
vector<shared_ptr<Counter>> counters { make_shared<Counter>(), make_shared<BackwardCounter>(3) };

for (auto& counter : counters) {
    for (int i = 0; i < 5; ++i) cout << (*counter)() << ' '; cout << '\n';
}

## Memory Management

### UndoRedoCounter

Consider implementing a counter that can undo and redo changes to its count. We need to modify `Counter` slightly to allow `set_count` to be overriden as well:

In [ ]:
%%cpp
class Counter {
protected:
    int count;
public:
    Counter() = default;
    explicit Counter(int count) : count{count} {}
    virtual ~Counter() = default;

    virtual int operator()() {
        return ++count;
    }
    virtual bool set_count(int count) {
        if (count>=0) return this->count=count, true; 
        else return cerr << "Count cannot go negative.\n", false;
    }
    operator int() const {
        return count;
    }
};

The following is an implementation that uses a [circular buffer](https://en.wikipedia.org/wiki/Circular_buffer) to keep a history of count values:

In [ ]:
%%cpp
class UndoRedoCounter : public Counter {
    int *buffer;
    size_t historysize, head, tail, current;

    size_t after(size_t pos) const { return (pos+1) % historysize; }
    size_t before(size_t pos) const { return (historysize+pos-1) % historysize; }

    int push(int value) {
        if (head == before(tail)) tail=after(tail);
        return buffer[head = current = after(current)] = value;
    }

public:
    UndoRedoCounter() : UndoRedoCounter(0) {}
    explicit UndoRedoCounter(int count, int historysize=5)
    : Counter(count), historysize(historysize), head(0), tail(0), current(-1) {
        if (historysize<1) throw runtime_error("History size must be at least 1.");
        buffer = new int[historysize];
        push(count);
    }
    ~UndoRedoCounter() override {
        delete[] buffer;
    }

    int operator()() override {
        return push(Counter::operator()());
    }
    bool set_count(int count) override {
        return Counter::set_count(count) && (push(count), true);
    }

    bool can_undo() const { return current != tail; }
    bool can_redo() const { return current != head; }
    int undo() {
        if (can_undo()) current = before(current); else cerr << "Cannot undo.\n";
        return count=buffer[current];
    }
    int redo() {
        if (can_redo()) current = after(current); else cerr << "Cannot redo.\n";
        return count=buffer[current];
    }
    void print_history() const {
        cout << "History Buffer [size=" << historysize << "]:\n";
        for (size_t i = 0; i < historysize; ++i) {
            cout << "[" << i << "] " << buffer[i];
            if (i == head) cout << " <- head";
            if (i == tail) cout << " <- tail";
            if (i == current) cout << " <- current";
            cout << "\n";
        }
    }
};

The derived class `UndoRedoCounter` used new private members to keep track of the history as follows:

::::{code} cpp
class UndoRedoCounter : public Counter {
    int *buffer;
    size_t historysize, head, tail, current;

    size_t after(size_t pos) const { return (pos+1) % historysize; }
    size_t before(size_t pos) const { return (historysize+pos-1) % historysize; }

    int push(int value) {
        if (head == before(tail)) tail=after(tail);
        return buffer[head = current = after(current)] = value;
    }
    ...
}
::::

- `int *buffer` is a dynamically allocated array that stores the history of counter values.
- `size_t historysize` is the size of the buffer.
- `size_t head` is the index of the most recent entry in the buffer.
- `size_t tail` is the index of the oldest entry still retained in the buffer.
- `size_t current` is the index of the current value.
- `size_t after(size_t pos) const` returns the wrapped index after `pos`.
- `size_t before(size_t pos) const` returns the wrapped index before `pos`.

::::{exercise}
:label: ex:push

Explain how `int push(int value)` works.

::::

YOUR ANSWER HERE

### Polymorphic Destruction

::::{code} cpp
class UndoRedoCounter : public Counter {
    ...

public:
    UndoRedoCounter() : UndoRedoCounter(0) {} // delegating constructor for code reuse
    explicit UndoRedoCounter(int count, int historysize=5)
    : Counter(count), historysize(historysize), head(0), tail(0), current(-1) {
        if (historysize<1) throw runtime_error("History size must be at least 1.");
        buffer = new int[historysize];
        push(count);
    }
    ~UndoRedoCounter() override {
        delete[] buffer;
    } // `Counter::~Counter()` called automatically after `~UndoRedoCounter()`
    ...
    void print_history() const {
        cout << "History Buffer [size=" << historysize << "]:\n";
        for (size_t i = 0; i < historysize; ++i) {
            cout << "[" << i << "] " << buffer[i];
            if (i == head) cout << " <- head";
            if (i == tail) cout << " <- tail";
            if (i == current) cout << " <- current";
            cout << "\n";
        }
    }
    ...
}
::::

- The default constructor `UndoRedoCounter() : UndoRedoCounter(0)` is a [delegating constructor](https://en.cppreference.com/w/cpp/language/constructor#Delegating_constructor) that reuse the code of the explicit constructor below, with the `count` default to `0`.
- The explicit constructor `explicit UndoRedoCounter(int count, int historysize=5)` initializes private members as follows:
    - The `count` is initialized by reusing the base class constructor `explicit Counter(int count)`.
    - Since the current count must be stored in the history as well, the constructor ensures `historysize` is at least 1 before allocating the memory for `buffer`.
    - `current` is initialized to `-1` since it will be incremented to `0` by `push(count);`.
    - `head` and `tail` is initialized to `0`, which will be the position of the current count after the push.
- The destructor `~UndoRedoCounter() override` releases the memory allocated to `buffer`. The destructor of the base class is also called [in reverse order of construction](https://en.cppreference.com/w/cpp/language/destructor.html#Destruction_sequence).
- The methd `void print_history()` prints the contents of the history `buffer` annotated with the position of `head`, `tail`, and `current`.

In [ ]:
%%cpp
UndoRedoCounter c;
c.print_history();

::::{exercise}
:label: ex:head

Why not initialize `head` to `-1` instead of `0`?

::::

YOUR ANSWER HERE

::::{exercise}
:label: ex:virtual_destructor

Explain how failing to declare the base‑class destructor `~Counter()` as `virtual` can result in a memory leak when an `UndoRedoCounter` object goes out of scope.

::::

YOUR ANSWER HERE

### Accessing Members

::::{code} cpp
class UndoRedoCounter : public Counter {
...

public:
    ...
    int operator()() override {
        return push(Counter::operator()());
    }
    bool set_count(int count) override {
        return Counter::set_count(count) && (push(count), true);
    }
    ...
};
::::

Both `int operator()()` and `bool set_count(int count)` reuse the corresponding methods they override, which can be accessed [using the scope resolution operator `::`](https://en.cppreference.com/w/cpp/language/qualified_lookup.html#Class_members) in addition to the [`using` declaration for inheriting constructors](https://en.cppreference.com/w/cpp/language/using_declaration.html#Inheriting_constructors).  The updated `count` is then pushed to the history buffer.

In [ ]:
%%cpp
UndoRedoCounter c;
cout << c << ' ' << c() << ' ';         // count = 1
cout << c() << ' ';                     // count = 2
if (c.set_count(10)) cout << c << '\n'; // count = 10
c.print_history();

::::{code} cpp
class UndoRedoCounter : public Counter {
...

public:
    ...
    bool can_undo() const { return current != tail; }
    ...
    int undo() {
        if (can_undo()) current = before(current); else cerr << "Cannot undo.\n";
        return count=buffer[current];
    }
    ...
};
::::

- `int undo()` moves `current` back by one position if doing so does not exceed the tail, i.e., the condition `current != tail` returned by `can_undo`, and
- `count` is then updated to the value of the `buffer` at the `current` position.

In [ ]:
%%cpp
UndoRedoCounter c;
cout << c << ' ' << c() << ' ';         // count = 1
cout << c() << ' ';                     // count = 2
if (c.set_count(10)) cout << c << '\n'; // count = 10
cout << c.undo() << '\n';        // count = 2
c.print_history();
cout << c.undo() << ' ';        // count = 1
cout << c.undo() << '\n';        // count = 0
c.print_history();
cout << c.undo() << '\n';       // cannot undo beyond the first change

In [ ]:
%%cpp
UndoRedoCounter c;
for (int i=0; i<6; i++) cout << c() << ' '; cout << '\n';      // count = 6
c.print_history();
for (int i=0; i<4; i++) cout << c.undo() << ' '; cout << '\n'; // count = 2
c.print_history();
cout << c.undo() << '\n';       // can undo at most `historysize-1` times

::::{code} cpp
class UndoRedoCounter : public Counter {
...

public:
    ...
    bool can_redo() const { return current != head; }
    ...
    int redo() {
        if (can_redo()) current = after(current); else cerr << "Cannot redo.\n";
        return count=buffer[current];
    }
    ...
};
::::

- `int redo()` moves `current` forward by one position if doing so does not exceed the head, i.e., the condition `current != head` returned by `can_redo`, and
- `count` is then updated to the value of the `buffer` at the `current` position.

In [ ]:
%%cpp
UndoRedoCounter c;
cout << c << ' ' << c() << ' ';             // count = 1
if (c.set_count(10)) cout << c << '\n'; // count = 10
cout << c.undo() << ' ';        // count = 1
cout << c.undo() << ' ';        // count = 0
cout << c.redo() << '\n';        // count = 1
c.print_history();
cout << c.redo() << '\n';        // count = 10
c.print_history();
cout << c.redo() << '\n';       // cannot redo after the last change

Note that we cannot access new methods defined in the derived class through a base class pointer:

In [ ]:
%%cpp
unique_ptr<Counter> c { make_unique<UndoRedoCounter>() };
cout << *c << ' ' << (*c)() << ' ';   // count = 1
// cout << (*c).undo() << ' ';        // error: no member named 'undo' in 'Counter'

::::{exercise}
:label: ex:dynamic_cast

Why `undo` cannot be found even though the actual type of `c` defines it?

::::

::::{solution} ex:dynamic_cast
:class: dropdown

This is because `undo` is not defined in `Counter` so the compiler cannot resolve it statically, unlike other virtual methods defined in the base class. 

::::

 To fix the issue, we can use [`dynamic_cast`](https://en.cppreference.com/w/cpp/language/dynamic_cast.html):

In [ ]:
%%cpp
unique_ptr<Counter> c { make_unique<UndoRedoCounter>() };
cout << *c << ' ' << (*c)() << ' ';   // count = 1

UndoRedoCounter *raw=dynamic_cast<UndoRedoCounter*>(c.get());
if (raw) {
    unique_ptr<UndoRedoCounter> d(raw);
    c.release();
    cout << d->undo() << ' ';
}

In [ ]:
%%ai
Explain in a paragraph or two how dynamic_cast can be used to convert a derived
class accessed through its base class pointer back to the derived class?

### Rule of Three

`UndoRedoCounter` appears to fail:

In [ ]:
%%cpp
UndoRedoCounter c1, c2=c1;               // c1: 0    c2: 0
if (c2.set_count(10)) cout << c2 << ' '; //              10         
cout << c1() << ' ';                     //     1
cout << c2() << ' ';                     //              11
cout << c1.undo() << ' ';                //     0      
cout << c2.undo() << '\n';                //              10?

Shouldn't `c2.undo()` return the count `10` set by `c2.set_count(10)`?

It works if we directly initialize the count of `c2` to `10`:

In [ ]:
%%cpp
UndoRedoCounter c1, c2(10);              // c1: 0    c2: 10        
cout << c1() << ' ';                     //     1
cout << c2() << ' ';                     //              11
cout << c1.undo() << ' ';                //     0
cout << c2.undo() << '\n';                //              10

But the above is not a fix. There is an even bigger memory issue:

(crash:r3_1)=
::::{caution} 😈: Want to see a crash?

Execute the follow code:

```cpp
{
    UndoRedoCounter c1, c2=c1;               // c1: 0    c2: 0
    if (c2.set_count(10)) cout << c2 << ' '; //              10         
    cout << c1() << ' ';                     //     1
    cout << c2() << ' ';                     //              11
    cout << c1.undo() << ' ';                //     0      
    cout << c2.undo() << '\n';                //              10?
}
```

::::

 Let us figure out what is wrong before quickly jumping to a fix. We can print the history of `c2`:

In [ ]:
%%cpp
UndoRedoCounter c1, c2=c1;                // c1: 0    c2: 0
if (c2.set_count(10)) cout << c2 << '\n'; //              10
c2.print_history();
cout << c1() << '\n';                     //     1
c2.print_history();

The history for `c2` is modified `c1()` even though `c2` is a separate copy. This is a mutation bug due to how [the default copy constructor works](https://en.cppreference.com/w/cpp/language/copy_constructor.html#Implicitly-defined_copy_constructor): The constructor perform a *shallow copy* of the `buffer` pointer, meaning both objects will point to the same memory.

::::{exercise}
:label: ex:r3_1

Explains the [crash](#crash:r3_1).

::::

::::{solution} ex:r3_1
:class: dropdown

Note that `c1` and `c2` are destroyed when they go out of the block scope of the compound expression.
When `c2` is destroyed, it executes `delete[] buffer`, leaving `c1` with a dangling pointer `buffer`. When `c1` is destroyed, `delete[] buffer` is deleted again. Double deletion leads to undefined behavior and memory corruption that cause the crash.

::::

To fix the issue, we should either:
1. Delete the copy constructor if copying is not intended:
   :::::{code} cpp
   UndoRedoCounter(const UndoRedoCounter&) = delete;
   :::::
2. Or, define the copy constructor to perform a deep copy of the buffer.
    :::::{code} cpp
    UndoRedoCounter(const UndoRedoCounter& other)
        : Counter(other), historysize(other.historysize),
          head(other.head), tail(other.tail), current(other.current) {
        buffer = new int[historysize];
        for (size_t i = 0; i < historysize; ++i)
            buffer[i] = other.buffer[i];
    }
    :::::
    This constructor:
    - Copies the base `Counter` part using its copy constructor.
    - Allocates a new buffer of the same size.
    - Copies each element from the source buffer.

We will go with the second fix:

In [ ]:
%%cpp
class UndoRedoCounter : public Counter {
    int *buffer;
    size_t historysize, head, tail, current;

    size_t after(size_t pos) const { return (pos+1) % historysize; }
    size_t before(size_t pos) const { return (historysize+pos-1) % historysize; }

    int push(int value) {
        if (head == before(tail)) tail=after(tail);
        return buffer[head = current = after(current)] = value;
    }

public:
    UndoRedoCounter() : UndoRedoCounter(0) {}
    explicit UndoRedoCounter(int count, int historysize=5)
    : Counter(count), historysize(historysize), head(0), tail(0), current(-1) {
        if (historysize<1) throw runtime_error("History size must be at least 1.");
        buffer = new int[historysize];
        push(count);
    }
    UndoRedoCounter(const UndoRedoCounter& other)
        : Counter(other), historysize(other.historysize),
          head(other.head), tail(other.tail), current(other.current) {
        buffer = new int[historysize];
        for (size_t i = 0; i < historysize; ++i)
            buffer[i] = other.buffer[i];
    }
    ~UndoRedoCounter() override {
        delete[] buffer;
    }

    int operator()() override {
        return push(Counter::operator()());
    }
    bool set_count(int count) override {
        return Counter::set_count(count) && (push(count), true);
    }

    bool can_undo() const { return current != tail; }
    bool can_redo() const { return current != head; }
    int undo() {
        if (can_undo()) current = before(current); else cerr << "Cannot undo.\n";
        return count=buffer[current];
    }
    int redo() {
        if (can_redo()) current = after(current); else cerr << "Cannot redo.\n";
        return count=buffer[current];
    }
    void print_history() const {
        cout << "History Buffer [size=" << historysize << "]:\n";
        for (size_t i = 0; i < historysize; ++i) {
            cout << "[" << i << "] " << buffer[i];
            if (i == head) cout << " <- head";
            if (i == tail) cout << " <- tail";
            if (i == current) cout << " <- current";
            cout << "\n";
        }
    }
};

It appears to work:

In [ ]:
%%cpp
{
    UndoRedoCounter c1, c2=c1;               // c1: 0    c2: 0
    if (c2.set_count(10)) cout << c2 << ' '; //              10         
    cout << c1() << ' ';                     //     1
    cout << c2() << ' ';                     //              11
    cout << c1.undo() << ' ';                //     0      
    cout << c2.undo() << '\n';                //              10?
}

Not really...

In [ ]:
%%cpp
UndoRedoCounter c1, c2;                  // c1: 0    c2: 0
c2=c1;
if (c2.set_count(10)) cout << c2 << ' '; //              10         
cout << c1() << ' ';                     //     1
cout << c2() << ' ';                     //              11
cout << c1.undo() << ' ';                //     0      
cout << c2.undo() << '\n';                //              10?

There is again a mutation bug because the [default copy assignment operator](https://en.cppreference.com/w/cpp/language/as_operator.html#Implicitly-defined_copy_assignment_operator) for `c2=c1` performs a *shallow* copy of the `buffer`.

::::{exercise}
:label: ex:r3_2

Explain how the above code leads to memory leak.

::::

YOUR ANSWER HERE

To fix the issue, we should either:
1. Delete the copy assignment operator if copying is not intended:
   :::::{code} cpp
   UndoRedoCounter& operator=(const UndoRedoCounter&) = delete;
   :::::
2. Or, define the copy assignment operator to perform a deep copy of the buffer.
    :::::{code} cpp
    UndoRedoCounter& operator=(const UndoRedoCounter& other) {
       if (this == &other) return *this; // self-assignment check
        Counter::operator=(other);       // assign base class instance
        delete[] buffer;                 // free existing buffer
        historysize = other.historysize;
        head = other.head;
        tail = other.tail;
        current = other.current;
        buffer = new int[historysize];
        for (size_t i = 0; i < historysize; ++i)
            buffer[i] = other.buffer[i];
        return *this;
    }
    :::::
    This operator:
    - Checks for self-assignment.
    - Assigns the base class part.
    - Deletes the old buffer and allocates a new one.
    - Copies the buffer contents.

In [ ]:
%%cpp
class UndoRedoCounter : public Counter {
    int *buffer;
    size_t historysize, head, tail, current;

    size_t after(size_t pos) const { return (pos+1) % historysize; }
    size_t before(size_t pos) const { return (historysize+pos-1) % historysize; }

    int push(int value) {
        if (head == before(tail)) tail=after(tail);
        return buffer[head = current = after(current)] = value;
    }

public:
    UndoRedoCounter() : UndoRedoCounter(0) {}
    explicit UndoRedoCounter(int count, int historysize=5)
    : Counter(count), historysize(historysize), head(0), tail(0), current(-1) {
        if (historysize<1) throw runtime_error("History size must be at least 1.");
        buffer = new int[historysize];
        push(count);
    }
    UndoRedoCounter(const UndoRedoCounter& other)
        : Counter(other), historysize(other.historysize),
          head(other.head), tail(other.tail), current(other.current) {
        buffer = new int[historysize];
        for (size_t i = 0; i < historysize; ++i)
            buffer[i] = other.buffer[i];
    }
    ~UndoRedoCounter() override {
        delete[] buffer;
    }

    UndoRedoCounter& operator=(const UndoRedoCounter& other) {
       if (this == &other) return *this; // self-assignment check
        Counter::operator=(other); // assign base class part
        delete[] buffer; // free existing buffer
        historysize = other.historysize;
        head = other.head;
        tail = other.tail;
        current = other.current;
        buffer = new int[historysize];
        for (size_t i = 0; i < historysize; ++i)
            buffer[i] = other.buffer[i];
        return *this;
    }
    int operator()() override {
        return push(Counter::operator()());
    }
    bool set_count(int count) override {
        return Counter::set_count(count) && (push(count), true);
    }

    bool can_undo() const { return current != tail; }
    bool can_redo() const { return current != head; }
    int undo() {
        if (can_undo()) current = before(current); else cerr << "Cannot undo.\n";
        return count=buffer[current];
    }
    int redo() {
        if (can_redo()) current = after(current); else cerr << "Cannot redo.\n";
        return count=buffer[current];
    }
    void print_history() const {
        cout << "History Buffer [size=" << historysize << "]:\n";
        for (size_t i = 0; i < historysize; ++i) {
            cout << "[" << i << "] " << buffer[i];
            if (i == head) cout << " <- head";
            if (i == tail) cout << " <- tail";
            if (i == current) cout << " <- current";
            cout << "\n";
        }
    }
};

It appears to work now:

In [ ]:
%%cpp
UndoRedoCounter c1, c2;                  // c1: 0    c2: 0
c2=c1;
if (c2.set_count(10)) cout << c2 << ' '; //              10         
cout << c1() << ' ';                     //     1
cout << c2() << ' ';                     //              11
cout << c1.undo() << ' ';                //     0      
cout << c2.undo() << '\n';                //              10?

::::{exercise}
:label: ex:r3_3

To improve code reuse between the copy constructor and assignment operator, add a private method `void copy_from(const UndoRedoCounter& other)` to handle buffer allocation and copying, which both the copy constructor and assignment operator can reuse.

::::

In [ ]:
%%cpp
class UndoRedoCounter : public Counter {
    int *buffer;
    size_t historysize, head, tail, current;

    size_t after(size_t pos) const { return (pos+1) % historysize; }
    size_t before(size_t pos) const { return (historysize+pos-1) % historysize; }

    int push(int value) {
        if (head == before(tail)) tail=after(tail);
        return buffer[head = current = after(current)] = value;
    }
    void copy_from(const UndoRedoCounter& other) {
        /*
        # REPLACE THE ENTIRE COMMENT WITH YOUR CODE #
        */
    }

public:
    UndoRedoCounter() : UndoRedoCounter(0) {}
    explicit UndoRedoCounter(int count, int historysize=5)
    : Counter(count), historysize(historysize), head(0), tail(0), current(-1) {
        if (historysize<1) throw runtime_error("History size must be at least 1.");
        buffer = new int[historysize];
        push(count);
    }
    UndoRedoCounter(const UndoRedoCounter& other)
        : Counter(other) {
        copy_from(other);
    }
    ~UndoRedoCounter() override {
        delete[] buffer;
    }

    UndoRedoCounter& operator=(const UndoRedoCounter& other) {
        if (this == &other) return *this;
        Counter::operator=(other);
        delete[] buffer;
        copy_from(other);
        return *this;
    }
    int operator()() override {
        return push(Counter::operator()());
    }
    bool set_count(int count) override {
        return Counter::set_count(count) && (push(count), true);
    }

    bool can_undo() const { return current != tail; }
    bool can_redo() const { return current != head; }
    int undo() {
        if (can_undo()) current = before(current); else cerr << "Cannot undo.\n";
        return count=buffer[current];
    }
    int redo() {
        if (can_redo()) current = after(current); else cerr << "Cannot redo.\n";
        return count=buffer[current];
    }
    void print_history() const {
        cout << "History Buffer [size=" << historysize << "]:\n";
        for (size_t i = 0; i < historysize; ++i) {
            cout << "[" << i << "] " << buffer[i];
            if (i == head) cout << " <- head";
            if (i == tail) cout << " <- tail";
            if (i == current) cout << " <- current";
            cout << "\n";
        }
    }
};

In [ ]:
%%cpp
// tests
{
    UndoRedoCounter c1, c2=c1;               // c1: 0    c2: 0
    if (c2.set_count(10)) cout << c2 << ' '; //              10         
    cout << c1() << ' ';                     //     1
    cout << c2() << ' ';                     //              11
    cout << c1.undo() << ' ';                //     0      
    cout << c2.undo() << '\n';                //              10?
}
{
    UndoRedoCounter c1, c2;                  // c1: 0    c2: 0
    c2=c1;
    if (c2.set_count(10)) cout << c2 << ' '; //              10         
    cout << c1() << ' ';                     //     1
    cout << c2() << ' ';                     //              11
    cout << c1.undo() << ' ';                //     0      
    cout << c2.undo() << '\n';                //              10?
}

The issue is summarized into the following rule of thumb for C++ programming:

::::{admonition} Rule of Three

The Rule of Three states that if a class declares any one of the following, it should explicitly provide the other two as well:

1. A destructor  
2. A copy constructor  
3. A copy assignment operator 

In other words, whenever you define one of these special member functions you should typically define all three to avoid subtle resource‑management bugs.

::::

### Rule of Five

Note that there is no memory corruption for the move semantics:

In [ ]:
%%cpp
// tests
{
    UndoRedoCounter c1, c2=std::move(c1);    // c1: 0    c2: 0
    if (c2.set_count(10)) cout << c2 << ' '; //              10         
    cout << c1() << ' ';                     //     1
    cout << c2() << ' ';                     //              11
    cout << c1.undo() << ' ';                //     0      
    cout << c2.undo() << '\n';               //              10?
}
{
    UndoRedoCounter c1, c2;                  // c1: 0    c2: 0
    c2=std::move(c1);
    if (c2.set_count(10)) cout << c2 << ' '; //              10         
    cout << c1() << ' ';                     //     1
    cout << c2() << ' ';                     //              11
    cout << c1.undo() << ' ';                //     0      
    cout << c2.undo() << '\n';               //              10?
}

::::{exercise}
:label: ex:r5_1

Why does the above code appear to run but `c1` is not actually moved?

:::{hint}
:class: dropdown

The [default move constructor](https://en.cppreference.com/w/cpp/language/move_constructor.html#Implicitly-declared_move_constructor) and the [default move assignment operator](https://en.cppreference.com/w/cpp/language/move_assignment.html#Implicitly-declared_move_assignment) are not declared.

:::

::::

YOUR ANSWER HERE

::::{admonition} Rule of Five

If a class needs a *destructor*, *copy constructor*, or *copy assignment operator*, it likely also needs a *move constructor* and *move assignment operator* for efficient transfer of temporary objects.

::::

The move constructor can be defined as follows to transfer the memory allocated to the buffer:

```cpp
UndoRedoCounter(UndoRedoCounter&& other) noexcept
    : Counter(std::move(other)),
      buffer(other.buffer),      // transfer allocated memory
      historysize(other.historysize),
      head(other.head),
      tail(other.tail),
      current(other.current) {
    other.buffer = nullptr;      // double deletion is okay for nullptr
    other.historysize = 0;       // prevent access to buffer
}
```

Move Assignment Operator

```cpp
UndoRedoCounter& operator=(UndoRedoCounter&& other) noexcept {
    if (this == &other) return *this;     // self-move check
    Counter::operator=(std::move(other)); // move base class instance
    delete[] buffer;                      // free existing buffer
    buffer = other.buffer;
    historysize = other.historysize;
    head = other.head;
    tail = other.tail;
    current = other.current;
    other.buffer = nullptr;
    other.historysize = 0;
    return *this;
}
```

In [ ]:
%%cpp
class UndoRedoCounter : public Counter {
    int *buffer;
    size_t historysize, head, tail, current;

    size_t after(size_t pos) const { return (pos+1) % historysize; }
    size_t before(size_t pos) const { return (historysize+pos-1) % historysize; }

    int push(int value) {
        if (head == before(tail)) tail=after(tail);
        return buffer[head = current = after(current)] = value;
    }

public:
    UndoRedoCounter() : UndoRedoCounter(0) {}
    explicit UndoRedoCounter(int count, int historysize=5)
    : Counter(count), historysize(historysize), head(0), tail(0), current(-1) {
        if (historysize<1) throw runtime_error("History size must be at least 1.");
        buffer = new int[historysize];
        push(count);
    }
    UndoRedoCounter(const UndoRedoCounter& other)
        : Counter(other), historysize(other.historysize),
          head(other.head), tail(other.tail), current(other.current) {
        buffer = new int[historysize];
        for (size_t i = 0; i < historysize; ++i)
            buffer[i] = other.buffer[i];
    }
    UndoRedoCounter(UndoRedoCounter&& other) noexcept
        : Counter(std::move(other)),
          buffer(other.buffer),
          historysize(other.historysize),
          head(other.head),
          tail(other.tail),
          current(other.current) {
        other.buffer = nullptr;
        other.historysize = 0;
    }
    ~UndoRedoCounter() override {
        delete[] buffer;
    }

    UndoRedoCounter& operator=(const UndoRedoCounter& other) {
       if (this == &other) return *this; // self-assignment check
        Counter::operator=(other); // assign base class part
        delete[] buffer; // free existing buffer
        historysize = other.historysize;
        head = other.head;
        tail = other.tail;
        current = other.current;
        buffer = new int[historysize];
        for (size_t i = 0; i < historysize; ++i)
            buffer[i] = other.buffer[i];
        return *this;
    }
    UndoRedoCounter& operator=(UndoRedoCounter&& other) noexcept {
        if (this == &other) return *this;
        Counter::operator=(std::move(other));
        delete[] buffer;
        buffer = other.buffer;
        historysize = other.historysize;
        head = other.head;
        tail = other.tail;
        current = other.current;
        other.buffer = nullptr;
        other.historysize = 0;
        return *this;
    }
    int operator()() override {
        return push(Counter::operator()());
    }
    bool set_count(int count) override {
        return Counter::set_count(count) && (push(count), true);
    }

    bool can_undo() const { return current != tail; }
    bool can_redo() const { return current != head; }
    int undo() {
        if (can_undo()) current = before(current); else cerr << "Cannot undo.\n";
        return count=buffer[current];
    }
    int redo() {
        if (can_redo()) current = after(current); else cerr << "Cannot redo.\n";
        return count=buffer[current];
    }
    void print_history() const {
        cout << "History Buffer [size=" << historysize << "]:\n";
        for (size_t i = 0; i < historysize; ++i) {
            cout << "[" << i << "] " << buffer[i];
            if (i == head) cout << " <- head";
            if (i == tail) cout << " <- tail";
            if (i == current) cout << " <- current";
            cout << "\n";
        }
    }
};

With these additions, the class now fully supports:
- Polymorphic deletion with virtual destructor;
- Deep copying by the copy constructor/assignment; and
- Efficient transfer of resource by the move constructor/assignment.

In [ ]:
%%cpp
{
    UndoRedoCounter c1(10);   
    UndoRedoCounter c2=std::move(c1);
    cout << "c1:\n";
    c1.print_history();
    cout << "c2:\n";
    c2.print_history();
}
{
    UndoRedoCounter c1(10), c2;
    c2=std::move(c1);
    cout << "c1:\n";
    c1.print_history();
    cout << "c2:\n";
    c2.print_history();
}

::::{exercise}
:label: ex:vector

We can simplify `UndoRedoCounter` class by using standard containers like `std::vector` to manage the history buffer. This approach eliminates the need to manually define destructors, copy/move constructors, and assignment operators, allowing us to rely on the compiler-generated defaults. Complete the following code for such an implementation.

::::

In [ ]:
%%cpp
class UndoRedoCounter : public Counter {
    vector<int> buffer;
    size_t head, tail, current;

    size_t after(size_t pos) const { return (pos + 1) % buffer.size(); }
    size_t before(size_t pos) const { return (buffer.size() + pos - 1) % buffer.size(); }

    int push(int value) {
        if (head == before(tail)) tail = after(tail);
        current = after(current);
        head = current;
        buffer[current] = value;
        return value;
    }

public:
    /*
    # REPLACE THE ENTIRE COMMENT WITH YOUR CODE #
    */
};

// test
{
    UndoRedoCounter c1(10);   
    UndoRedoCounter c2=std::move(c1);
    cout << "c1:\n";
    c1.print_history();
    cout << "c2:\n";
    c2.print_history();
}
{
    UndoRedoCounter c1(10), c2;
    c2=std::move(c1);
    cout << "c1:\n";
    c1.print_history();
    cout << "c2:\n";
    c2.print_history();
}
/* Expect
c1:
History Buffer [size=0]:
c2:
History Buffer [size=5]:
[0] 10 <- head <- tail <- current
[1] 0
[2] 0
[3] 0
[4] 0
c1:
History Buffer [size=0]:
c2:
History Buffer [size=5]:
[0] 10 <- head <- tail <- current
[1] 0
[2] 0
[3] 0
[4] 0
*/

::::{exercise}

Make `Counter` an [abstract base class](https://en.cppreference.com/w/cpp/language/abstract_class.html) by setting one or more of the virtual method to `0`. Then implement `ForwardCounter` and `BackwardCounter` as the derived class of `Counter`, and `UndoRedoCounter` as the derived class of `ForwardCounter`.

::::